### <span style="color:blue"> *---------------------------------------*</span>
# <span style="color:blue"> *UDV project*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
# Imports libraries

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer

import pickle
import time
import matplotlib.pyplot as plt

In [ ]:
# Set the global seed for reproducibility

public_seed = 529
torch.manual_seed(public_seed)
print("Current Seed is {0}".format(public_seed))

In [ ]:
# Obtain device

from udvFunctions.udvDevice import get_device
device = get_device()
device = "cpu"

In [ ]:
# Load dataset

from udvFunctions.udvDatasetPreprocessing import HousingPriceDataset
data_path = "./HP_Orig.csv"
dataset = HousingPriceDataset(data_path)

In [ ]:
# Prepare UDV constraints

from udvFunctions.udvConstraints import Matrix_bothside, UDV_Diag
from udvFunctions.udvConstraints import Vector_left_U, Vector_right_V

In [ ]:
# Prepare the single-connection layer that carry weight matrix 'w'

from udvFunctions.udvDiagonalLayer import D_singleConnection   

In [ ]:
# Prepare networks

from udvFunctions.udvRegNetworks import UDV_net_1, UDV_relu_1
from udvFunctions.udvRegNetworks import relu_net_1, fc_net_1

In [ ]:
# Prepare loss function (optional)

from udvFunctions.udvLoss import UDV_Loss

In [ ]:
# Prepare training framework

# Model_0: Matrix_uwv
from udvFunctions.udvRegMatrixUWV import udv_frame_m_uwv

# Model_1: Matrix_uv
from udvFunctions.udvRegMatrixUV import udv_frame_m_uv

# Model_4-5: ReLU,UV_ReLU
from udvFunctions.udvRegReLU import udv_frame_relu, uvrelu_frame_m_uv

In [ ]:
# Identical initialisation

from udvFunctions.udvSameInit import seedList1

In [ ]:
# Other customised functions

from udvFunctions.udvOtherFunctions import store_metrics, take_avg, check_shapes

### <span style="color:blue"> *---------------------------------------*</span>
# <span style="color:blue"> *Setting Experiments:*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

# Dataset split
training_ratio = 0.8
train_size = int(training_ratio * len(dataset))
validation_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, validation_size])

# Set number of epochs and how many different seeds for training 
num_epochs = 100         # Number of epochs; Min:5; 200
num_seeds = 50         # Number of seeds;  Min:1; 1000

num_input = len(dataset.features[0,:])
num_output = 1                                                                                  # Number of output feature
num_hidden_1 = round((((num_output+2)*num_input)**0.5) + (2*((num_input/(num_output+2))**0.5))) # Number of hidde neurons in fully connected layer 1 (constrained layer)
num_hidden_2 = round(num_output*((num_input/(num_output+2))**0.5))                              # Discarded (Expandable design)

# Load data
batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

# Load data (Full batch)
# batch_size = "full"
# train_dataloader = DataLoader(train_dataset, batch_size = len(train_dataset), shuffle = True)
# val_dataloader = DataLoader(val_dataset, batch_size = len(val_dataset), shuffle = False)

# Set optimiser (later) and loss function
optimiser_name = 'Adam'             # 'Adam', 'NAdam', 'SGD', 'SGDM'
SGD_M = 0.9                         # Only available when 'SGDM' is specified in optimiser_name
learning_rate = 1e-3

# loss_fn = UDV_Loss()    # MSE/2
loss_fn = nn.MSELoss()  # MSE

# Constraints setting
vector_u_norm = 1
vector_v_norm = 1
matrix_uvnorm = 1
d_threshold = 0
d_boundto = 0

# Fully Connect network without any activation leads many NaN with high learning rate
if learning_rate >= 1:
    test_LinearAct = False
else:
    test_LinearAct = True

# time record 
time_record = []

# Set result path
result_path = './{0}_{1}_H1_{2}_H2_{3}_BS{4}_E{5}_S{6}/SingleLayer'.format(optimiser_name, learning_rate, num_hidden_1, num_hidden_2, batch_size, num_epochs, num_seeds)
if not os.path.exists(result_path):
    os.makedirs(result_path)  

# Re-producible setting
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with uwv constraints (Matrix): Model_0_UDV*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_ulist1, init_wlist1, init_ulist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )

full_train_loss_model_0 = []
full_val_loss_model_0 = []

u_1_M_model_0 = []
w_1_M_model_0 = []
u_2_M_model_0 = []

constraints_uv = Matrix_bothside(normLim = matrix_uvnorm)
constraints_d = UDV_Diag(threshold = d_threshold, boundTo = d_boundto)

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1):
    model = UDV_net_1(num_input = num_input, num_hidden_1 = num_hidden_1, num_output = num_output)
    with torch.no_grad():
        check_shapes(model_weight = model.fc1.weight, list_sample = init_ulist1[i-1])
        check_shapes(model_weight = model.diag1.weight, list_sample = init_wlist1[i-1])
        check_shapes(model_weight = model.fc2.weight, list_sample = init_ulist2[i-1])
        model.fc1.weight = nn.Parameter(init_ulist1[i-1])
        model.diag1.weight = nn.Parameter(init_wlist1[i-1])
        model.fc2.weight = nn.Parameter(init_ulist2[i-1])
    
    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, save_weights_list = udv_frame_m_uwv(model = model, 
                                                                         optimizer = optimizer, 
                                                                         loss_fn = loss_fn, 
                                                                         train_loader = train_dataloader, 
                                                                         val_loader = val_dataloader, 
                                                                         num_epochs = num_epochs, 
                                                                         device = device,
                                                                         constraints_uv = constraints_uv,
                                                                         constraints_d = constraints_d,
                                                                        )  
    full_train_loss_model_0 = store_metrics(full_train_loss_model_0, train_losses)
    full_val_loss_model_0 = store_metrics(full_val_loss_model_0, val_losses)
    
    check_shapes(model_weight = model.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.diag1.weight, list_sample = save_weights_list[1])
    check_shapes(model_weight = model.fc2.weight, list_sample = save_weights_list[2])
    u_1_M_model_0.append(save_weights_list[0])
    w_1_M_model_0.append(save_weights_list[1])
    u_2_M_model_0.append(save_weights_list[2])

avg_train_loss_model_0 = take_avg(full_train_loss_model_0)
avg_val_loss_model_0 = take_avg(full_val_loss_model_0)
 
end_time = time.time()    
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_ulist1, init_wlist1, init_ulist2, model, constraints_uv, constraints_d, save_weights_list

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with uv constraints (Matrix): Model_1_UDV-s*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_ulist1, init_wlist1, init_ulist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )

full_train_loss_model_1 = []
full_val_loss_model_1 = []

u_1_M_model_1 = []
w_1_M_model_1 = []
u_2_M_model_1 = []

constraints_uv = Matrix_bothside(normLim = matrix_uvnorm)

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1):
    model = UDV_net_1(num_input = num_input, num_hidden_1 = num_hidden_1, num_output = num_output)
    with torch.no_grad():
        check_shapes(model_weight = model.fc1.weight, list_sample = init_ulist1[i-1])
        check_shapes(model_weight = model.diag1.weight, list_sample = init_wlist1[i-1])
        check_shapes(model_weight = model.fc2.weight, list_sample = init_ulist2[i-1])
        model.fc1.weight = nn.Parameter(init_ulist1[i-1])
        model.diag1.weight = nn.Parameter(init_wlist1[i-1])
        model.fc2.weight = nn.Parameter(init_ulist2[i-1])

    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, save_weights_list = udv_frame_m_uv(model = model, 
                                                                        optimizer = optimizer, 
                                                                        loss_fn = loss_fn, 
                                                                        train_loader = train_dataloader, 
                                                                        val_loader = val_dataloader, 
                                                                        num_epochs = num_epochs, 
                                                                        device = device,
                                                                        constraints_uv = constraints_uv,
                                                                       )  
    full_train_loss_model_1 = store_metrics(full_train_loss_model_1, train_losses)
    full_val_loss_model_1 = store_metrics(full_val_loss_model_1, val_losses)
    
    check_shapes(model_weight = model.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.diag1.weight, list_sample = save_weights_list[1])
    check_shapes(model_weight = model.fc2.weight, list_sample = save_weights_list[2])
    u_1_M_model_1.append(save_weights_list[0])
    w_1_M_model_1.append(save_weights_list[1])
    u_2_M_model_1.append(save_weights_list[2])

avg_train_loss_model_1 = take_avg(full_train_loss_model_1)
avg_val_loss_model_1 = take_avg(full_val_loss_model_1)

end_time = time.time()    
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_ulist1, init_wlist1, init_ulist2, model, constraints_uv, save_weights_list

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with ReLU_Diag_uwv constraints (Matrix): Model_2_UDV_ReLU*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_ulist1, init_wlist1, init_ulist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )

full_train_loss_model_2 = []
full_val_loss_model_2 = []

u_1_M_model_2 = []
w_1_M_model_2 = []
u_2_M_model_2 = []

constraints_uv = Matrix_bothside(normLim = matrix_uvnorm)
constraints_d = UDV_Diag(threshold = d_threshold, boundTo = d_boundto)

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1):
    model = UDV_relu_1(num_input = num_input, num_hidden_1 = num_hidden_1, num_output = num_output)
    with torch.no_grad():
        check_shapes(model_weight = model.fc1.weight, list_sample = init_ulist1[i-1])
        check_shapes(model_weight = model.diag1.weight, list_sample = init_wlist1[i-1])
        check_shapes(model_weight = model.fc2.weight, list_sample = init_ulist2[i-1])
        model.fc1.weight = nn.Parameter(init_ulist1[i-1])
        model.diag1.weight = nn.Parameter(init_wlist1[i-1])
        model.fc2.weight = nn.Parameter(init_ulist2[i-1])
    
    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, save_weights_list = udv_frame_m_uwv(model = model, 
                                                                         optimizer = optimizer, 
                                                                         loss_fn = loss_fn, 
                                                                         train_loader = train_dataloader, 
                                                                         val_loader = val_dataloader, 
                                                                         num_epochs = num_epochs, 
                                                                         device = device,
                                                                         constraints_uv = constraints_uv,
                                                                         constraints_d = constraints_d,
                                                                        )  
    full_train_loss_model_2 = store_metrics(full_train_loss_model_2, train_losses)
    full_val_loss_model_2 = store_metrics(full_val_loss_model_2, val_losses)
    
    check_shapes(model_weight = model.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.diag1.weight, list_sample = save_weights_list[1])
    check_shapes(model_weight = model.fc2.weight, list_sample = save_weights_list[2])
    u_1_M_model_2.append(save_weights_list[0])
    w_1_M_model_2.append(save_weights_list[1])
    u_2_M_model_2.append(save_weights_list[2])

avg_train_loss_model_2 = take_avg(full_train_loss_model_2)
avg_val_loss_model_2 = take_avg(full_val_loss_model_2)
 
end_time = time.time()    
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_ulist1, init_wlist1, init_ulist2, model, constraints_uv, constraints_d, save_weights_list

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with ReLU_Diag_uv constraints (Matrix): Model_3_UDV_ReLU-s*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_ulist1, init_wlist1, init_ulist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )

full_train_loss_model_3 = []
full_val_loss_model_3 = []

u_1_M_model_3 = []
w_1_M_model_3 = []
u_2_M_model_3 = []

constraints_uv = Matrix_bothside(normLim = matrix_uvnorm)

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1):
    model = UDV_relu_1(num_input = num_input, num_hidden_1 = num_hidden_1, num_output = num_output)
    with torch.no_grad():
        check_shapes(model_weight = model.fc1.weight, list_sample = init_ulist1[i-1])
        check_shapes(model_weight = model.diag1.weight, list_sample = init_wlist1[i-1])
        check_shapes(model_weight = model.fc2.weight, list_sample = init_ulist2[i-1])
        model.fc1.weight = nn.Parameter(init_ulist1[i-1])
        model.diag1.weight = nn.Parameter(init_wlist1[i-1])
        model.fc2.weight = nn.Parameter(init_ulist2[i-1])

    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, save_weights_list = udv_frame_m_uv(model = model, 
                                                                        optimizer = optimizer, 
                                                                        loss_fn = loss_fn, 
                                                                        train_loader = train_dataloader, 
                                                                        val_loader = val_dataloader, 
                                                                        num_epochs = num_epochs, 
                                                                        device = device,
                                                                        constraints_uv = constraints_uv,
                                                                       )  
    full_train_loss_model_3 = store_metrics(full_train_loss_model_3, train_losses)
    full_val_loss_model_3 = store_metrics(full_val_loss_model_3, val_losses)
    
    check_shapes(model_weight = model.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.diag1.weight, list_sample = save_weights_list[1])
    check_shapes(model_weight = model.fc2.weight, list_sample = save_weights_list[2])
    u_1_M_model_3.append(save_weights_list[0])
    w_1_M_model_3.append(save_weights_list[1])
    u_2_M_model_3.append(save_weights_list[2])

avg_train_loss_model_3 = take_avg(full_train_loss_model_3)
avg_val_loss_model_3 = take_avg(full_val_loss_model_3)

end_time = time.time()    
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_ulist1, init_wlist1, init_ulist2, model, constraints_uv, save_weights_list

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with ReLU: Model_4_UV_ReLU*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_llist1, init_wlist1, init_llist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )

full_train_loss_model_4 = []
full_val_loss_model_4 = []

l_1_model_4 = []
l_2_model_4 = []

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1):
    # Load relu model
    model = relu_net_1(num_input = num_input, num_hidden_1 = num_hidden_1, num_output = num_output)
    with torch.no_grad():
        check_shapes(model_weight = model.fc1.weight, list_sample = init_llist1[i-1])
        check_shapes(model_weight = model.fc2.weight, list_sample = init_llist2[i-1])
        model.fc1.weight = nn.Parameter(init_llist1[i-1])
        model.fc2.weight = nn.Parameter(init_llist2[i-1])

    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, save_weights_list = udv_frame_relu(model = model, 
                                                                        optimizer = optimizer, 
                                                                        loss_fn = loss_fn, 
                                                                        train_loader = train_dataloader, 
                                                                        val_loader = val_dataloader, 
                                                                        num_epochs = num_epochs, 
                                                                        device = device
                                                                       )  
    full_train_loss_model_4 = store_metrics(full_train_loss_model_4, train_losses)
    full_val_loss_model_4 = store_metrics(full_val_loss_model_4, val_losses)
    
    check_shapes(model_weight = model.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.fc2.weight, list_sample = save_weights_list[1])
    l_1_model_4.append(save_weights_list[0])
    l_2_model_4.append(save_weights_list[1])
        
avg_train_loss_model_4 = take_avg(full_train_loss_model_4)
avg_val_loss_model_4 = take_avg(full_val_loss_model_4)

end_time = time.time()    
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_llist1, init_wlist1, init_llist2, model, save_weights_list

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with ReLU_uv constraints (Matrix): Model_5_UV_ReLU(constrained)*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_llist1, init_wlist1, init_llist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )

full_train_loss_model_5 = []
full_val_loss_model_5 = []

l_1_model_5 = []
l_2_model_5 = []

constraints_uv = Matrix_bothside(normLim = matrix_uvnorm)

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1):
    model = relu_net_1(num_input = num_input, num_hidden_1 = num_hidden_1, num_output = num_output)
    with torch.no_grad():
        check_shapes(model_weight = model.fc1.weight, list_sample = init_llist1[i-1])
        check_shapes(model_weight = model.fc2.weight, list_sample = init_llist2[i-1])
        model.fc1.weight = nn.Parameter(init_llist1[i-1])
        model.fc2.weight = nn.Parameter(init_llist2[i-1])

    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, save_weights_list = uvrelu_frame_m_uv(model = model, 
                                                                           optimizer = optimizer, 
                                                                           loss_fn = loss_fn, 
                                                                           train_loader = train_dataloader, 
                                                                           val_loader = val_dataloader, 
                                                                           num_epochs = num_epochs, 
                                                                           device = device,
                                                                           constraints_uv = constraints_uv,
                                                                          )  
    full_train_loss_model_5 = store_metrics(full_train_loss_model_5, train_losses)
    full_val_loss_model_5 = store_metrics(full_val_loss_model_5, val_losses)
    
    check_shapes(model_weight = model.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.fc2.weight, list_sample = save_weights_list[1])
    l_1_model_5.append(save_weights_list[0])
    l_2_model_5.append(save_weights_list[1])

avg_train_loss_model_5 = take_avg(full_train_loss_model_5)
avg_val_loss_model_5 = take_avg(full_val_loss_model_5)

end_time = time.time()    
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_llist1, init_wlist1, init_llist2, model, constraints_uv, save_weights_list

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *Saving*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
# Save variables for analysis

store_file_path = '{0}/results.pkl'.format(result_path)

variables = {
    'device': device,
    'data_path': data_path,
    'public_seed': public_seed,
    
    'num_input': num_input,
    'num_hidden_1': num_hidden_1,
    'num_hidden_2': num_hidden_2,
    'num_output': num_output,
    
    'batch_size': batch_size,
    'num_epochs': num_epochs,
    'num_seeds': num_seeds,
    'optimiser_name': optimiser_name,
    'learning_rate': learning_rate,
    
    'full_train_loss_model_0': full_train_loss_model_0,
    'full_train_loss_model_1': full_train_loss_model_1,
    'full_train_loss_model_2': full_train_loss_model_2,
    'full_train_loss_model_3': full_train_loss_model_3,
    'full_train_loss_model_4': full_train_loss_model_4,
    'full_train_loss_model_5': full_train_loss_model_5,
    'avg_train_loss_model_0': avg_train_loss_model_0,
    'avg_train_loss_model_1': avg_train_loss_model_1,
    'avg_train_loss_model_2': avg_train_loss_model_2,
    'avg_train_loss_model_3': avg_train_loss_model_3,
    'avg_train_loss_model_4': avg_train_loss_model_4,
    'avg_train_loss_model_5': avg_train_loss_model_5,
    'full_val_loss_model_0': full_val_loss_model_0,
    'full_val_loss_model_1': full_val_loss_model_1,
    'full_val_loss_model_2': full_val_loss_model_2,
    'full_val_loss_model_3': full_val_loss_model_3,
    'full_val_loss_model_4': full_val_loss_model_4,
    'full_val_loss_model_5': full_val_loss_model_5,
    'avg_val_loss_model_0': avg_val_loss_model_0,
    'avg_val_loss_model_1': avg_val_loss_model_1,
    'avg_val_loss_model_2': avg_val_loss_model_2,
    'avg_val_loss_model_3': avg_val_loss_model_3,
    'avg_val_loss_model_4': avg_val_loss_model_4,
    'avg_val_loss_model_5': avg_val_loss_model_5,
    
    'u_1_M_model_0': u_1_M_model_0,
    'w_1_M_model_0': w_1_M_model_0,
    'u_2_M_model_0': u_2_M_model_0,
    
    'u_1_M_model_1': u_1_M_model_1,
    'w_1_M_model_1': w_1_M_model_1,
    'u_2_M_model_1': u_2_M_model_1,
    
    'u_1_M_model_2': u_1_M_model_2,
    'w_1_M_model_2': w_1_M_model_2,
    'u_2_M_model_2': u_2_M_model_2,

    'u_1_M_model_3': u_1_M_model_3,
    'w_1_M_model_3': w_1_M_model_3,
    'u_2_M_model_3': u_2_M_model_3,

    'l_1_model_4': l_1_model_4,
    'l_2_model_4': l_2_model_4,
    
    'l_1_model_5': l_1_model_5,
    'l_2_model_5': l_2_model_5,
    
    'time_record': time_record
}

with open(store_file_path, 'wb') as file:
    pickle.dump(variables, file)

print("Saving pickle file, done")

from udvFunctions.udvOtherFunctions import avg_stable

last_n = 20           # how many epochs (from end) are averaged
final_train_loss_m0 = avg_stable(avg_train_loss_model_0, last_n)
final_train_loss_m1 = avg_stable(avg_train_loss_model_1, last_n)
final_train_loss_m2 = avg_stable(avg_train_loss_model_2, last_n)
final_train_loss_m3 = avg_stable(avg_train_loss_model_3, last_n)
final_train_loss_m4 = avg_stable(avg_train_loss_model_4, last_n)
final_train_loss_m5 = avg_stable(avg_train_loss_model_5, last_n)

final_val_loss_m0 = avg_stable(avg_val_loss_model_0, last_n)
final_val_loss_m1 = avg_stable(avg_val_loss_model_1, last_n)
final_val_loss_m2 = avg_stable(avg_val_loss_model_2, last_n)
final_val_loss_m3 = avg_stable(avg_val_loss_model_3, last_n)
final_val_loss_m4 = avg_stable(avg_val_loss_model_4, last_n)
final_val_loss_m5 = avg_stable(avg_val_loss_model_5, last_n)

print('Batch size: {0}\n#Seeds: {1}\n#Epochs: {2}\nOptimier: {3}\nLearning Rate: {4}\n#Hidden_1: {5}\n#Hidden_2: {6}'.format(batch_size, num_seeds, num_epochs, optimiser_name, learning_rate, num_hidden_1, num_hidden_2))
print('Final results are averaged from the last {0} epochs\n'.format(last_n))
print('model_0: UDV                      (avg:-{0}): T-loss {1}; V-loss is {2}'.format(last_n, final_train_loss_m0, final_val_loss_m0))
print('model_1: UDV-s                    (avg:-{0}): T-loss {1}; V-loss is {2}'.format(last_n, final_train_loss_m1, final_val_loss_m1))
print('model_2: UDV-ReLU                 (avg:-{0}): T-loss {1}; V-loss is {2}'.format(last_n, final_train_loss_m2, final_val_loss_m2))
print('model_3: UDV-ReLU-s               (avg:-{0}): T-loss {1}; V-loss is {2}'.format(last_n, final_train_loss_m3, final_val_loss_m3))
print('model_4: UV-ReLU                  (avg:-{0}): T-loss {1}; V-loss is {2}'.format(last_n, final_train_loss_m4, final_val_loss_m4))
print('model_5: UV-ReLU(Constrained)     (avg:-{0}): T-loss {1}; V-loss is {2}'.format(last_n, final_train_loss_m5, final_val_loss_m5))